In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import pickle
from collections import Counter
from sklearn.model_selection import train_test_split
from PIL import Image

In [2]:
path = "/media/yesindeed/WD5T/data/brazilian-ophthalmological/1.0.0/"

metadata = pd.read_csv(path + "labels.csv")
metadata

,image_id,patient_id,camera,patient_age,comorbidities,diabetes_time_y,insuline,patient_sex,exam_eye,diabetes,...,amd,vascular_occlusion,hypertensive_retinopathy,drusens,hemorrhage,retinal_detachment,myopic_fundus,increased_cup_disc,other,quality
0,img00001,1,Canon CR,48.0,diabetes1,12,yes,1,1,yes,...,0,0,0,0,0,0,0,1,0,Adequate
1,img00002,1,Canon CR,48.0,diabetes1,12,yes,1,2,yes,...,0,0,0,0,0,0,0,1,0,Adequate
2,img00003,2,Canon CR,18.0,diabetes1,7,yes,2,1,yes,...,0,0,0,0,0,0,0,0,0,Adequate
3,img00004,2,Canon CR,18.0,diabetes1,7,yes,2,2,yes,...,0,0,0,0,0,0,0,0,0,Adequate
4,img00005,3,Canon CR,22.0,diabetes1,11,yes,1,1,yes,...,0,0,0,0,0,0,0,0,0,Adequate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16261,img16262,8522,Canon CR,33.0,diabetes1,14,yes,1,2,yes,...,0,0,0,0,0,0,0,1,0,Adequate
16262,img16263,8523,Canon CR,19.0,diabetes1,4,yes,1,1,yes,...,0,0,0,0,0,0,0,0,0,Adequate
16263,img16264,8523,Canon CR,19.0,diabetes1,4,yes,1,2,yes,...,0,0,0,0,0,0,0,0,0,Adequate
16264,img16265,8524,Canon CR,20.0,diabetes1,NaN,yes,1,1,yes,...,0,0,0,0,0,0,0,1,0,Adequate


In [14]:
metadata["patient_sex"].value_counts()

patient_sex
2    10052
1     6214
Name: count, dtype: int64

In [11]:
tem_train = pd.read_csv("/home/yesindeed/Documents/Dropbox/BREST/split/new_train.csv")
tem_test = pd.read_csv("/home/yesindeed/Documents/Dropbox/BREST/split/new_test.csv")

df = metadata.copy(deep=True)
df_train = df.loc[df["image_id"].isin(tem_train["image_id"])].reset_index(drop=True)
df_test = df.loc[df["image_id"].isin(tem_test["image_id"])].reset_index(drop=True)

# assert len(df_train) == len(tem_train), f"{len(df_train)}, {len(tem_train)}"

df_train.to_csv(os.path.join(path, "train.csv"), index=False)
df_test.to_csv(os.path.join(path, "test.csv"), index=False)

In [4]:
from tqdm import tqdm

# resize all images to save storage
low_res_path = os.path.join(path, "small")

if not os.path.exists(low_res_path):
    os.makedirs(low_res_path)

for i in tqdm(range(len(metadata))):
    item = metadata.iloc[i]

    img_id = item["image_id"]

    if os.path.exists(os.path.join(low_res_path, f"{img_id}.jpg")):
        continue

    img = Image.open(os.path.join(path, "fundus_photos", f"{img_id}.jpg"))
    img = img.resize(size=(256, 256), resample=Image.BICUBIC)
    img.save(os.path.join(low_res_path, f"{img_id}.jpg"))

100%|██████████| 16266/16266 [1:42:14<00:00,  2.65it/s]  


In [3]:
# age

df_test = pd.read_csv(os.path.join(path, "test.csv"))

df_test = df_test[~df_test["patient_age"].isnull()]

df_test["age_binary"] = df_test["patient_age"].values.astype("float")
df_test["age_binary"] = np.where(
    df_test["age_binary"].between(-1, 60), 0, df_test["age_binary"])
df_test["age_binary"] = np.where(
    df_test["age_binary"] >= 60, 1, df_test["age_binary"])

class_counts = df_test["age_binary"].value_counts()
print(class_counts)
min_count = class_counts.min()
balanced_test_meta = df_test.groupby("age_binary").apply(
    lambda x: x.sample(min_count)).reset_index(drop=True)


balanced_test_meta.to_csv(os.path.join(path, "test_age.csv"), index=False)

balanced_test_meta["age_binary"].value_counts()

age_binary
1.0    853
0.0    770
Name: count, dtype: int64


/tmp/ipykernel_13391/3512247386.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_test_meta = df_test.groupby("age_binary").apply(


age_binary
0.0    770
1.0    770
Name: count, dtype: int64